### Using IPUMS records, calculate in-migration and out-migration to/from NYC - NYC Metro and domestic U.S.

Requires download of csv extracts from IPUMS USA website
https://usa.ipums.org

For more information about specific variables available for download, refer to IPUMS-USA website, ex.:
https://usa.ipums.org/usa-action/variables/MIGRATE1#codes_section

This notebook updates the previous to pull 4 1-year Estimates for 2016 to 2019

In [1]:
import pandas as pd
import numpy as np
import math

#### Stat functions for using replicate weights

In [2]:
# functions to calculate standard error, moe, and coefficient of variation
def get_se(per_wt,rep_weights):
    result = math.sqrt((sum(map(lambda x: (x-per_wt)**2,rep_weights))/20))
    return result

def get_moe(se):
    return se*1.645 #90% confidence interval

def agg_moe(m):
    result = math.sqrt(sum(map(lambda x: x**2, m)))
    return result

def get_cv(est,m):
    if est == 0:
        return 0
    else:
        return (np.absolute(m/1.645/est))*100

In [3]:
# create a list of replicate weights
repwt = 'REPWTP'
repwts = [repwt+str(i) for i in range(1, 81)]

#### Geography look up files & cleanup

In [5]:
# pull in geography reference files
df_migpuma = pd.read_csv('../data/migpwpuma_xwalk_10.csv')
df_respuma = pd.read_csv('../data/respuma_xwalk_10.csv')
df_migpl = pd.read_csv('../data/migpl_xwalk.csv')

#df_migpuma.head()

In [6]:
# clean up migpuma table for merge with data table 
df_migpuma['MIGPUMA_str'] = df_migpuma['MIGPUMAID'].apply(str).apply(lambda x: '{0:0>8}'.format(x))
df_migpuma['CountyFIP'] = df_migpuma['CountyFIP'].apply(str)
df_migpuma = df_migpuma.drop(columns=['STATEFIP','State','MIGSTATE','MIGPUMA','PWSTATE','PWPUMA','StringGIS'])
#df_migpuma.head()

In [7]:
df_respuma['puma_id'] = df_respuma['GEOID10'].apply(str).apply(lambda x: '{0:0>8}'.format(x))
df_respuma['CountyFIPS'] = df_respuma['CountyFIPS'].apply(str).apply(lambda x: '{0:0>5}'.format(x))
df_respuma = df_respuma.drop(columns=['StateFIPS','PUMA'])

In [8]:
#forgot to pull COUNTYFIPS in pums extract for in-migration
#these are to clean the in-migration table for borough totals
nyc_county = dict(df_respuma[['puma_id','CountyFIPS']].values)
nyc_respuma = df_respuma[df_respuma['Subregion']=='NYC']['puma_id'].tolist()

### In-migrants to NYC boroughs by NYC Metro Subregion & County
Following options available:
- By NYC total or Borough of Current Residence (i.e. destination)
- BY NYC Metro Subregion of residence 1 year ago (i.e. origin)

In [10]:
# read in full inflow table (raw PUMS data)
df = pd.read_csv(f'../data/nys_mig_inout_2021.csv.gz',compression='gzip')
#df.head()

In [11]:
# create a new id columns for merge with geo lookup tables
df['migpuma_id'] = df['MIGPLAC1'].apply(str).apply(lambda x: '{0:0>3}'.format(x)) + \
                    df['MIGPUMA1'].apply(str).apply(lambda x: '{0:0>5}'.format(x))

df['in_respuma'] = df['STATEFIP'].apply(str).apply(lambda x: '{0:0>3}'.format(x)) + \
                   df['PUMA'].apply(str).apply(lambda x: '{0:0>5}'.format(x))

In [12]:
# REDUCE DATA TABLE TO PEOPLE WHO HAVE MOVED TO NYC COUNTIES ONLY IN THE LAST YEAR
# Select only records for people who have moved in the last year
# MIGRATE1 == 2,3,4 ; other codes are people who haven't moved
mig_codes = [2,3,4]
df = df[df['MIGRATE1'].isin(mig_codes)]

# Select records for people who live in NYC currently
df = df[df['in_respuma'].isin(nyc_respuma)]
df['in_stco'] = df.in_respuma.map(nyc_county)
df.head()

,YEAR,SAMPLE,SERIAL,CBSERIAL,HHWT,HHTYPE,REPWT,CLUSTER,STATEFIP,PUMA,...,REPWTP74,REPWTP75,REPWTP76,REPWTP77,REPWTP78,REPWTP79,REPWTP80,migpuma_id,in_respuma,in_stco
2,2021,202101,843587,2021010000099,109.0,0,1,2021008435871,36,3710,...,111,112,107,108,109,25,25,03603700,03603710,36005
14,2021,202101,843634,2021010000662,68.0,0,1,2021008436341,36,3704,...,68,9,11,69,67,67,10,03603700,03603704,36005
17,2021,202101,843639,2021010000701,69.0,0,1,2021008436391,36,3703,...,69,69,130,69,70,72,70,03604100,03603703,36005
21,2021,202101,843654,2021010000866,24.0,0,1,2021008436541,36,3802,...,25,25,4,46,25,26,45,03603800,03603802,36061
25,2021,202101,843665,2021010000978,22.0,0,1,2021008436651,36,4009,...,1,22,23,3,23,3,22,03604000,03604009,36047


In [13]:
# merge with geography look up tables 
df_in = df.merge(df_migpl,how='left',left_on='MIGPLAC1',right_on='migplac_id').merge(df_migpuma,how='left',left_on='migpuma_id',right_on='MIGPUMA_str')
#df_in.head()

In [14]:
# replace NaN values in aggregation columns with domestic/intl values
column_clean = ['Subregion5','Subregion7','Region','County','CountyFIP']
for i in column_clean:
    df_in.loc[df_in[i].isnull(),i] = df_in['US_intl']

In [15]:
# reduce table to just counties and subregions of residence 1 year ago
cols = ['YEAR','in_stco','CountyFIP','Subregion7','Subregion5','Region','PERWT'] + repwts
df_in = df_in[cols]

# rename for clarity
df_in = df_in.rename(columns={'County':'out_co_name','CountyFIP':'out_stco',\
                              'Subregion5':'out_subreg_5','Subregion7':'out_subreg_7',\
                              'Region':'out_reg','PERWT':'in_pop'})
    
df_in.head()

,YEAR,in_stco,out_stco,out_subreg_7,out_subreg_5,out_reg,in_pop,REPWTP1,REPWTP2,REPWTP3,...,REPWTP71,REPWTP72,REPWTP73,REPWTP74,REPWTP75,REPWTP76,REPWTP77,REPWTP78,REPWTP79,REPWTP80
0,2021,36005,36005,NYC,NYC,NYC,109.0,109,202,26,...,104,192,193,111,112,107,108,109,25,25
1,2021,36005,36005,NYC,NYC,NYC,68.0,10,69,10,...,67,11,11,68,9,11,69,67,67,10
2,2021,36005,36081,NYC,NYC,NYC,69.0,10,127,129,...,69,73,131,69,69,130,69,70,72,70
3,2021,36061,36061,NYC,NYC,NYC,24.0,23,4,46,...,45,25,23,25,25,4,46,25,26,45
4,2021,36047,36047,NYC,NYC,NYC,22.0,44,43,24,...,45,1,21,1,22,23,3,23,3,22


#### NYC Boroughs by Subregion of Origin (simplified) table

Select different summary columns in groupby function to choose counties or aggregation by borough of current residence.

In [16]:
#CHANGE TO PIVOT TABLE AND THEN RESET INDEX

dff_in = df_in.groupby(['YEAR','in_stco','out_reg']).sum().reset_index()

In [17]:
# calculate standard error, margin of error, cv
# drop replicate weight columns
dff_in['in_se'] = dff_in.apply(lambda x: (get_se(x['in_pop'],x[repwts])),axis=1)
dff_in['in_moe'] = dff_in.apply(lambda x: (get_moe(x['in_se'])),axis=1)
dff_in['in_cv'] = dff_in.apply(lambda x: (get_cv(x['in_pop'],x['in_se'])),axis=1)

dff_in = dff_in.drop(columns=repwts) 
                            
dff_in

,YEAR,in_stco,out_reg,in_pop,in_se,in_moe,in_cv
0,2021,36005,NYC,77437.0,5772.019724,9494.972447,4.531202
1,2021,36005,Region,4538.0,1007.001639,1656.517695,13.489622
2,2021,36005,US,2308.0,670.910352,1103.647529,17.671068
3,2021,36047,NYC,226099.0,8916.872465,14668.255204,2.397441
4,2021,36047,Region,6690.0,1383.098189,2275.196521,12.567850
5,2021,36047,US,3291.0,820.310490,1349.410756,15.152507
6,2021,36061,NYC,166699.0,8532.327051,14035.677998,3.111491
7,2021,36061,Region,21978.0,2983.095867,4907.192701,8.251124
8,2021,36061,US,5908.0,992.163066,1632.108244,10.208846
9,2021,36081,NYC,138950.0,7721.864542,12702.467171,3.378296


### NYC resident out-migration to NYC Metro and U.S.

##### update csv paths & keep aggregation geography consistent with previous for net flow table in following section

In [18]:
# pull in outflow table
dff = pd.read_csv(f'../data/nys_mig_inout_2021.csv.gz',compression='gzip')

In [19]:
nyc_mig = {'03603700':'36005','03603800':'36061','03603900':'36085','03604000':'36047','03604100':'36081'}

In [20]:
# create new id columns for merge with geo lookup table
dff['in_respuma'] = dff['STATEFIP'].apply(str).apply(lambda x: '{0:0>3}'.format(x)) + \
                    dff['PUMA'].apply(str).apply(lambda x: '{0:0>5}'.format(x))

dff['out_migpuma'] = dff['MIGPLAC1'].apply(str).apply(lambda x: '{0:0>3}'.format(x)) + \
                    dff['MIGPUMA1'].apply(str).apply(lambda x: '{0:0>5}'.format(x))

In [21]:
# Select records only for people who lived in NYC one year ago (NYC out-migrants)
dff = dff[dff['out_migpuma'].isin(nyc_mig.keys())]
dff['out_stco'] = dff.out_migpuma.map(nyc_mig)

In [22]:
# merge with geography look up tables 
df_out = dff.merge(df_respuma,how='left',left_on='in_respuma',right_on='puma_id')

In [23]:
# replace NaN values in aggregation columns with US for non region
column_clean = ['NAME','Subregion','Subregion2','Region','County','CountyFIPS']
for i in column_clean:
    df_out.loc[df_out[i].isnull(),i] = 'US'

In [24]:
df_out.head()

,YEAR,SAMPLE,SERIAL,CBSERIAL,HHWT,HHTYPE,REPWT,CLUSTER,STATEFIP,PUMA,...,out_migpuma,out_stco,GEOID10,NAME,Subregion,Subregion2,Region,County,CountyFIPS,puma_id
0,2021,202101,843587,2021010000099,109.0,0,1,2021008435871,36,3710,...,03603700,36005,3603710.0,NYC-Bronx Community District 1 & 2--Hunts Poin...,NYC,NYC,NYC,Bronx,36005,03603710
1,2021,202101,843604,2021010000261,73.0,0,1,2021008436041,36,402,...,03603700,36005,NaN,US,US,US,US,US,US,NaN
2,2021,202101,843634,2021010000662,68.0,0,1,2021008436341,36,3704,...,03603700,36005,3603704.0,NYC-Bronx Community District 11--Pelham Parkwa...,NYC,NYC,NYC,Bronx,36005,03603704
3,2021,202101,843636,2021010000671,87.0,0,1,2021008436361,36,2001,...,03603900,36085,NaN,US,US,US,US,US,US,NaN
4,2021,202101,843637,2021010000679,78.0,0,1,2021008436371,36,906,...,03603700,36005,NaN,US,US,US,US,US,US,NaN


In [25]:
# reduce table to just current counties and subregions of residence 
cols = ['YEAR','County','CountyFIPS','Subregion','Subregion2','Region','out_stco','PERWT'] + repwts
df_out = df_out[cols]

# rename for clarity
df_out = df_out.rename(columns={'County':'in_co_name','CountyFIPS':'in_stco',\
                              'Subregion':'in_subreg_7','Subregion2':'in_subreg_5',\
                              'Region':'in_reg','PERWT':'out_pop'})
    
df_out.head()

,YEAR,in_co_name,in_stco,in_subreg_7,in_subreg_5,in_reg,out_stco,out_pop,REPWTP1,REPWTP2,...,REPWTP71,REPWTP72,REPWTP73,REPWTP74,REPWTP75,REPWTP76,REPWTP77,REPWTP78,REPWTP79,REPWTP80
0,2021,Bronx,36005,NYC,NYC,NYC,36005,109.0,109,202,...,104,192,193,111,112,107,108,109,25,25
1,2021,US,US,US,US,US,36005,73.0,135,139,...,72,139,76,13,72,74,11,73,134,139
2,2021,Bronx,36005,NYC,NYC,NYC,36005,68.0,10,69,...,67,11,11,68,9,11,69,67,67,10
3,2021,US,US,US,US,US,36085,87.0,85,85,...,162,164,87,87,88,85,13,158,91,13
4,2021,US,US,US,US,US,36005,78.0,12,76,...,79,79,79,11,79,145,77,144,147,78


#### NYC total outflow by Subregion Destination (simplified) table

Select different summary columns in groupby function to choose counties or aggregation by borough of previous residence.

In [26]:
dff_out = df_out.groupby(['YEAR','out_stco','in_reg']).sum().reset_index()
#dff_out

In [27]:
# calculate standard error, margin of error, cv
# drop replicate weight columns
dff_out['out_se'] = dff_out.apply(lambda x: (get_se(x['out_pop'],x[repwts])),axis=1)
dff_out['out_moe'] = dff_out.apply(lambda x: (get_moe(x['out_se'])),axis=1)
dff_out['out_cv'] = dff_out.apply(lambda x: (get_cv(x['out_pop'],x['out_se'])),axis=1)

dff_out = dff_out.drop(columns=repwts) 
                            
dff_out.head()

,YEAR,out_stco,in_reg,out_pop,out_se,out_moe,out_cv
0,2021,36005,NYC,72087.0,5540.177624,9113.592191,4.671978
1,2021,36005,Region,11146.0,1972.301067,3244.435256,10.756928
2,2021,36005,US,5371.0,883.330657,1453.078931,9.997749
3,2021,36047,NYC,224966.0,9519.322828,15659.286052,2.572310
4,2021,36047,Region,23059.0,4222.195779,6945.512057,11.130944


## Export in and out table to Excel

In [28]:
with pd.ExcelWriter(f'../output/nycmig_inout_reg_boro_2021.xlsx') as writer:
    dff_in.to_excel(writer,sheet_name="In_mig", index=False)
    dff_out.to_excel(writer,sheet_name="Out_mig", index=False)

## Pivot & resave

In [29]:
dfff_in = pd.pivot_table(dff_in,values=['in_pop','in_moe','in_cv'],index='in_stco',\
                         columns=['YEAR','out_reg'],aggfunc=np.sum)

In [30]:
dfff_in

in_cv                              in_moe               \
YEAR         2021                                2021                
out_reg       NYC     Region         US           NYC       Region   
in_stco                                                              
36005    4.531202  13.489622  17.671068   9494.972447  1656.517695   
36047    2.397441  12.567850  15.152507  14668.255204  2275.196521   
36061    3.111491   8.251124  10.208846  14035.677998  4907.192701   
36081    3.378296  10.252001  14.343101  12702.467171  3064.677626   
36085    6.898393  40.056135  46.022151   4841.717912   290.492981   

                        in_pop                   
YEAR                      2021                   
out_reg           US       NYC   Region      US  
in_stco                                          
36005    1103.647529   77437.0   4538.0  2308.0  
36047    1349.410756  226099.0   6690.0  3291.0  
36061    1632.108244  166699.0  21978.0  5908.0  
36081     903.949877  138950.0  11047.0  2329.0  
36085     384.819615   25937.0    268.0   309.0